<a href="https://colab.research.google.com/github/rishi02102017/fashion-mnist-wandb-cnn/blob/main/WandB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q wandb
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.nn.functional as F

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: m24csa013 (m24csa013-indian-institute-of-technology-jodhpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
class FashionCNN(nn.Module):
    def __init__(self, dropout_rate):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)       # (28-3+1)=26
        self.pool1 = nn.MaxPool2d(2, 2)        # 13x13
        self.conv2 = nn.Conv2d(32, 64, 3)      # (13-3+1)=11
        self.pool2 = nn.MaxPool2d(2, 2)        # 5x5
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

In [4]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    total_loss, correct = 0, 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = out.argmax(dim=1)
        correct += pred.eq(target).sum().item()
    return total_loss / len(train_loader), correct / len(train_loader.dataset)

def validate(model, device, val_loader, criterion):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            out = model(data)
            loss = criterion(out, target)
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    return total_loss / len(val_loader), correct / len(val_loader.dataset)

In [5]:
def main(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Dataset
        transform = transforms.Compose([transforms.ToTensor()])
        dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config.batch_size)

        # Model
        model = FashionCNN(config.dropout_rate).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

        for epoch in range(config.epochs):
            train_loss, train_acc = train(model, device, train_loader, optimizer, criterion, epoch)
            val_loss, val_acc = validate(model, device, val_loader, criterion)

            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_acc
            })

In [6]:
sweep_config = {
    "method": "random",
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "values": [0.01, 0.001, 0.0001]
        },
        "batch_size": {
            "values": [32, 64, 128]
        },
        "epochs": {
            "values": [5, 10, 15]
        },
        "dropout_rate": {
            "values": [0.3, 0.5, 0.7]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep=sweep_config, project="fashion-mnist-classification")

wandb.agent(sweep_id, function=main, count=10)

Create sweep with ID: 5d9b43dv
Sweep URL: https://wandb.ai/m24csa013-indian-institute-of-technology-jodhpur/fashion-mnist-classification/sweeps/5d9b43dv


wandb: Agent Starting Run: sucu7xji with config:
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0.5
wandb: 	epochs: 5
wandb: 	learning_rate: 0.01


100%|██████████| 26.4M/26.4M [00:03<00:00, 8.39MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 129kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.48MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 6.30MB/s]


epoch,▁▃▅▆█
train_accuracy,▁▆▇▇█
train_loss,█▃▂▁▁
val_accuracy,▁▅▆█▆
val_loss,█▃▃▁▂
epoch,5
train_accuracy,0.86696
train_loss,0.3607
val_accuracy,0.87367
val_loss,0.33148


wandb: Agent Starting Run: h2k905be with config:
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0.5
wandb: 	epochs: 15
wandb: 	learning_rate: 0.01
